Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [127]:
from random import random, choice, randint, sample
from dataclasses import dataclass
from copy import copy

import numpy as np
import lab9_lib

In [128]:
@dataclass
class Individual:
    def __init__(self, genotype, fitness):
        self.genotype = genotype
        self.fitness = fitness

In [129]:
@dataclass
class Population:
    def __init__(self, population_size, genotype_lenght, fitness, population= None):
        if population != None:
            self.population = population
        else:
            self.population = list()
            for _ in range(population_size):    
                genotype = np.random.randint(2, size=genotype_lenght)
                self.population.append(Individual(genotype, fitness(genotype)))

    def get_best_individual(self):
        sorted_population = sorted(self.population, key= lambda i: i.fitness, reverse=True)
        best_individual = sorted_population[0]
        return best_individual
    
    def select_parent(self, tournament_size):
        pool = [choice(self.population) for _ in range(tournament_size)]
        champion = max(pool, key= lambda i: i.fitness)
        return champion
    
    def survival_selection(self, size):
        sorted_population = sorted(self.population, key= lambda i: i.fitness, reverse=True)
        self.population = sorted_population[:size]

    def add_offspring(self, offspring: list[Individual]):
        self.population.extend(offspring)

## Mutation & Recombination

In [130]:
def mutate(individual: Individual, fitness) -> Individual:
    genotype_lenght = individual.genotype.shape[0]
    offspring = copy(individual)
    pos = randint(0, genotype_lenght - 1)
    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = fitness(offspring.genotype)
    return offspring

def one_cut_xover(individual_1: Individual, individual_2: Individual, fitness) -> Individual:
    genotype_lenght = individual_1.genotype.shape[0]
    cut_point = randint(0, genotype_lenght)
    new_genotype = np.concatenate((individual_1.genotype[:cut_point], individual_2.genotype[cut_point:]))
    offspring = Individual(new_genotype, fitness(new_genotype))
    return offspring

def two_cut_xover(individual_1: Individual, individual_2: Individual, fitness) -> Individual:
    genotype_lenght = individual_1.genotype.shape[0]
    cut_poits = sorted(sample(range(genotype_lenght), 2))
    new_genotype = np.concatenate((individual_1.genotype[:cut_poits[0]], individual_2.genotype[cut_poits[0]:cut_poits[1]], individual_1.genotype[cut_poits[1]:]))
    offspring = Individual(new_genotype, fitness(new_genotype))
    return offspring

def uniform_xover(individual_1: Individual, individual_2: Individual, fitness) -> Individual:
    genotype_lenght = individual_1.genotype.shape[0]
    new_genotype = np.array([choice([individual_1.genotype[i], individual_2.genotype[i]]) for i in range(genotype_lenght)])
    offspring = Individual(new_genotype, fitness(new_genotype))
    return offspring

def apply_xover(individual_1: Individual, individual_2: Individual, fitness) -> Individual:
    ## I choose randomly the xover function to apply
    xover_functions = [one_cut_xover, two_cut_xover, uniform_xover]
    f = choice(xover_functions)
    return f(individual_1, individual_2, fitness)

## Steady State EA

In [131]:
def steady_state_EA(population, num_generations, population_size, offspring_size, mutation_probability, tournament_size, fitness):
    for _ in range(num_generations):
        offspring = list()
        for _ in range(offspring_size):    
            if random() < mutation_probability:  # self-adapt mutation probability
                # mutation  # add more clever mutations
                p = population.select_parent(tournament_size)
                o = mutate(p, fitness)
            else:
                # xover # add more xovers
                p1 = population.select_parent(tournament_size)
                p2 = population.select_parent(tournament_size)
                o = apply_xover(p1, p2, fitness)
            offspring.append(o)
        population.add_offspring(offspring)
        population.survival_selection(population_size)

## Generational EA

In [132]:
def generational_EA(population, num_generations, population_size, offspring_size, mutation_probability, tournament_size, genotype_lenght, fitness):
    for _ in range(num_generations):
        new_generation = list()
        new_generation.append(population.get_best_individual()) ## Elitism, I add the best individual to avoid that the algorithm lose it
        for _ in range(offspring_size // 2):
            p = population.select_parent(tournament_size)
            if random() < mutation_probability:
                p = mutate(p, fitness)
            new_generation.append(p)

        population = Population(population_size, genotype_lenght, fitness, new_generation)
        offspring = list()

        for _ in range(offspring_size):    
            if random() < mutation_probability:  # self-adapt mutation probability
                # mutation  # add more clever mutations
                p = population.select_parent(tournament_size)
                o = mutate(p, fitness)
            else:
                # xover # add more xovers
                p1 = population.select_parent(tournament_size)
                p2 = population.select_parent(tournament_size)
                o = apply_xover(p1, p2, fitness)
            offspring.append(o)
        population.add_offspring(offspring)
        population.survival_selection(population_size)
    

## Tests

In [133]:
POPULATION_SIZE = 50
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = 0.3
GENOTYPE_LENGHT = 1000
num_generations = 2500
problem_instances = [1, 2, 5, 10]

In [137]:
## Steady State EA tests
offspring_size = 20
fitness = [lab9_lib.make_problem(1), lab9_lib.make_problem(2), lab9_lib.make_problem(5), lab9_lib.make_problem(10)]
print('Steady State EA:')
for i, f in enumerate(fitness):
    population = Population(POPULATION_SIZE, GENOTYPE_LENGHT, f)
    steady_state_EA(population, num_generations, POPULATION_SIZE, offspring_size, MUTATION_PROBABILITY, TOURNAMENT_SIZE, f)
    print('\tProblem instances = {0}\tgenerations = {1}\tfitness calls = {2}\tbest fitness = {3:.2%}'.format(problem_instances[i], num_generations, f.calls, population.get_best_individual().fitness))


Steady State EA:
	Problem instances = 1	generations = 2500	fitness calls = 50050	best fitness = 98.10%
	Problem instances = 2	generations = 2500	fitness calls = 50050	best fitness = 80.60%
	Problem instances = 5	generations = 2500	fitness calls = 50050	best fitness = 45.70%
	Problem instances = 10	generations = 2500	fitness calls = 50050	best fitness = 23.13%


In [138]:
## Generational EA tests
offspring_size = 40
fitness = [lab9_lib.make_problem(1), lab9_lib.make_problem(2), lab9_lib.make_problem(5), lab9_lib.make_problem(10)]
print('Generational EA:')
for i, f in enumerate(fitness):
    population = Population(POPULATION_SIZE, GENOTYPE_LENGHT, f)
    generational_EA(population, num_generations, POPULATION_SIZE, offspring_size, MUTATION_PROBABILITY, TOURNAMENT_SIZE, GENOTYPE_LENGHT, f)
    print('\tProblem instances = {0}\tgenerations = {1}\tfitness calls = {2}\tbest fitness = {3:.2%}'.format(problem_instances[i], num_generations, f.calls, population.get_best_individual().fitness))

Generational EA:
	Problem instances = 1	generations = 2500	fitness calls = 115128	best fitness = 53.60%
	Problem instances = 2	generations = 2500	fitness calls = 115089	best fitness = 50.00%
	Problem instances = 5	generations = 2500	fitness calls = 114930	best fitness = 19.11%
	Problem instances = 10	generations = 2500	fitness calls = 115116	best fitness = 10.99%
